In [15]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import re
from flair.data import Sentence
from flair.models import SequenceTagger

import numpy as np
import pandas as pd
import os
nltk.download('averaged_perceptron_tagger')
from nltk import sent_tokenize
from nltk import defaultdict
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk.chunk import ne_chunk

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\simiv\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-d

### Preprocessing

#### Bookwise preprocessing

In [2]:
nltk.__version__

'3.7'

In [3]:
books = ['Outlander', 'Dragonfly in Amber', 'Voyager', 'Drums of Autumn', 'The Fiery Cross', 'A Breath of Snow and Ashes',
         'An Echo in the Bone', 'Written in My Own Heart’s Blood']
extras = ['Other Books by this Author', 'About the Author']
bookstarts = [50, 17287, 37378, 61857, 89432, 119494, 152540, 177800, 202059]

In [4]:
f = open("data.txt", "r", encoding="utf8")
booknum = 1


book = ''
i = 0
for line in f:
    if i < bookstarts[booknum] and i > bookstarts[booknum-1]:
        book = book + line
    i = i + 1

In [5]:
# we want to segment the data into chapters as we want to see the topic per chapter. But how do we do that?
# the author devided the books into parts, this is how we can see when hte first chapter begins, then we search for a line
# that is written in all caps, after this the chapter starts: example: \n\n\n\n\n\n1\n\nA NEW BEGINNING\n\n
#
# book 1 has 41, 49, 63, 71, 111, 124, 103, 145
# found is 41, 49, 63, 71, 111, 124, 0, 0
def split_into_chapters(booknum, text):
    if booknum <7:
        return re.split(r'\n[0-9]+\n\n', book)
    else:
        # needs refinement
        return re.split(r'[A-Z ]+\n\n', book)

In [22]:
def split_into_paragraphs(text):
    splitted = re.split('(\*\*\*)', text)
    return [x for x in splitted if x != '']

In [28]:
# print(split_into_paragraphs(split_into_chapters(1, book)[4]))
sentences = split_into_paragraphs(split_into_chapters(1, book)[0])
sentences[0]


'Dragonfly in Amber\n\nVoyager\n\nDrums of Autumn\n\nThe Fiery Cross\n\nA Breath of Snow and Ashes\n\nAn Echo in the Bone\n\nWritten in My Own Heart’s Blood\n\nOther Books by this Author\n\nAbout the Author\n\n\n\n\n\nOUTLANDER\n\nA Delta Book\n\nPUBLISHING HISTORY\n\nDelacorte Press hardcover edition published 1991\n\nDelta trade paperback edition/July 2001\n\nPublished by\n\nBantam Dell\n\nA Division of Random House, Inc.\n\nNew York, New York\n\nAll rights reserved\n\nCopyright © 1991 by Diana Gabaldon\n\nTitle page art copyright © 1999 by Barbara Schnell\n\nLibrary of Congress Catalog Card Number: 90-019122\n\nNo part of this book may be reproduced or transmitted in any form or by any means, electronic or mechanical, including photocopying, recording, or by any information storage and retrieval system, without the written permission of the publisher, except where permitted by law.\n\nDelta is a registered trademark of Random House, Inc., and the colophon is a trademark of Random Ho

#### Linguistic Preprocessing

In [8]:
# tokenizer
per_paragraph = True

tokenizer = RegexpTokenizer(r"[a-zA-Z’]+")
chapters = split_into_chapters(booknum, book)
tchapters = []
for chapter in chapters:
    tparagraphs = []
    if per_paragraph:
        paragraphs = split_into_paragraphs(chapter)
        for paragraph in paragraphs:
            tparagraphs.append(tokenizer.tokenize(paragraph))
        tchapters.append(tparagraphs)
    else:
        tchapters.append(tokenizer.tokenize(chapter))

In [9]:
chapternum = 0
lemmatizer=WordNetLemmatizer()
if not per_paragraph:
    for word in tchapters[chapternum]:
        print(lemmatizer.lemmatize(word))
else:
    for paragraph in tchapters[chapternum]:
        for word in paragraph:
            print(lemmatizer.lemmatize(word))

Dragonfly
in
Amber
Voyager
Drums
of
Autumn
The
Fiery
Cross
A
Breath
of
Snow
and
Ashes
An
Echo
in
the
Bone
Written
in
My
Own
Heart’s
Blood
Other
Books
by
this
Author
About
the
Author
OUTLANDER
A
Delta
Book
PUBLISHING
HISTORY
Delacorte
Press
hardcover
edition
published
Delta
trade
paperback
edition
July
Published
by
Bantam
Dell
A
Division
of
Random
House
Inc
New
York
New
York
All
right
reserved
Copyright
by
Diana
Gabaldon
Title
page
art
copyright
by
Barbara
Schnell
Library
of
Congress
Catalog
Card
Number
No
part
of
this
book
may
be
reproduced
or
transmitted
in
any
form
or
by
any
mean
electronic
or
mechanical
including
photocopying
recording
or
by
any
information
storage
and
retrieval
system
without
the
written
permission
of
the
publisher
except
where
permitted
by
law
Delta
is
a
registered
trademark
of
Random
House
Inc
and
the
colophon
is
a
trademark
of
Random
House
Inc
Please
visit
our
website
at
www
bantamdell
com
eISBN
v
r
Contents
Master
Table
of
Contents
Outlander
Title
page
Copyrigh

In [18]:
tagger = SequenceTagger.load("flair/ner-english-ontonotes-fast")

for paragraph in tchapters[1]:
    sentence = Sentence(paragraph)
    tagger.predict(sentence)
    for entity in sentence.get_spans('ner'):
        if entity.tag=='PERSON' or entity.tag=='GPE'or entity.tag=='DATE':
            print(entity)


2023-05-14 12:27:36,546 SequenceTagger predicts: Dictionary with 75 tags: O, S-PERSON, B-PERSON, E-PERSON, I-PERSON, S-GPE, B-GPE, E-GPE, I-GPE, S-ORG, B-ORG, E-ORG, I-ORG, S-DATE, B-DATE, E-DATE, I-DATE, S-CARDINAL, B-CARDINAL, E-CARDINAL, I-CARDINAL, S-NORP, B-NORP, E-NORP, I-NORP, S-MONEY, B-MONEY, E-MONEY, I-MONEY, S-PERCENT, B-PERCENT, E-PERCENT, I-PERCENT, S-ORDINAL, B-ORDINAL, E-ORDINAL, I-ORDINAL, S-LOC, B-LOC, E-LOC, I-LOC, S-TIME, B-TIME, E-TIME, I-TIME, S-WORK_OF_ART, B-WORK_OF_ART, E-WORK_OF_ART, I-WORK_OF_ART, S-FAC
Span[14:15]: "Baird’s" → PERSON (0.9999)
Span[46:47]: "Baird" → PERSON (0.9999)
Span[57:58]: "Frank" → PERSON (0.9971)
Span[3:4]: "Baird" → PERSON (0.9828)
Span[3:4]: "Randall" → PERSON (0.9983)
Span[23:24]: "Ye" → PERSON (0.9835)
Span[1:2]: "I’ll" → PERSON (0.7898)
Span[5:6]: "Baird" → PERSON (0.9879)
Span[17:19]: "Frank We’ll" → PERSON (0.877)
Span[44:46]: "four years" → DATE (0.8341)
Span[32:33]: "Frank" → PERSON (0.9979)
Span[49:50]: "Baird" → PERSON (0.998